In [ ]:
from omero import client as om_client
from omero import model, grid
from omero import rtypes
from omero import ClientError
from omero import sys
from omero import constants

In [ ]:
USERNAME = "xxxx"
PASSWORD = "xxxxxxxx"
HOST = "xxxx.ac.uk"
PORT = 4064
CLIENT = om_client(HOST, PORT)
SESSION = None

In [ ]:
proj_ids, dataset_ids, annotation_ids = list(), list(), list()
params = sys.Parameters()
LINKED_ANNOS_BY_PROJ_QUERY = "select p from Project p left outer join  \\
                             fetch p.annotationLinks as links left outer join fetch  \\
                             links.child as annotation where p.id=:pid"
LINKED_ANNOS_BY_DS_QUERY = "select d from Dataset d left outer join  \\
                             fetch d.annotationLinks as links left outer join fetch  \\
                             links.child as annotation where d.id=:did"
LINKED_DATASETS_BY_PROJ_QUERY = "select p from Project p left outer join  \\
                                fetch p.datasetLinks as links left outer join fetch  \\
                                links.child as dataset where p.id=:pid"
# LINKED_DATASETS_BY_PROJ_QUERY = "select d from ProjectDatasetLink d where d.parent.id = :pid"
# LINKED_ANNOS_BY_DS_QUERY = "select a from DatasetAnnotationLink a where a.parent.id = :did"
ALL_PROJ_QUERY = "select p from Project p"

In [ ]:
def open_omero_session():
    global SESSION
    try:
        SESSION = CLIENT.getSession()
    except ClientError:
        print "No live session"
    SESSION = CLIENT.createSession(USERNAME, PASSWORD)

In [ ]:
def close_omero_session():
    CLIENT.closeSession()

In [ ]:
def destroy_omero_session():
    CLIENT.destroySession(CLIENT.getSessionId())

In [ ]:
def find_objects_by_query(query, params, all=False):
    global SESSION
    queryService = SESSION.getQueryService()
    # query = "\"select p from Project p left outer join fetch p.datasetLinks as links left outer join fetch links.child as dataset where p.id =:pid\""
    objects = list()
    if all == False:
        objects = queryService.findByQuery(query, params)
    else:
        objects = queryService.findAllByQuery(query, params)
    return objects

In [ ]:
def print_comment_divider():
    print "########################################################################################"
    print "###################--------------------------------------------#########################"
    print "########################################################################################"
try:
    open_omero_session()
except Exception as e:
    print "Existing session"
print SESSION

In [ ]:
%%timeit
## time to get projects
global params
params.map = dict()
all_projs = find_objects_by_query(ALL_PROJ_QUERY, params, True)
# print all_projs
global proj_ids
#     return all_projs
proj_ids = [i.id.val for i in all_projs]
proj_ids = list(set(proj_ids))  # remove any duplicates

In [ ]:
%%timeit
# Get LinkedDatasets for Projects
global dataset_ids
global proj_ids
for proj_id in proj_ids:
    params.map = "{\"pid\": rtypes.rlong(proj_id)}"
    proj_datasets = find_objects_by_query(LINKED_DATASETS_BY_PROJ_QUERY, params, True)
    ds_ids = [i.id.val for i in proj_datasets]
    ds_ids = list(set(ds_ids))  # remove any duplicates
#     print ds_ids
    dataset_ids.extend(ds_ids)

In [ ]:
%%timeit
# Get LinkedAnnotations for Datasets
global annotation_ids
for ds_id in dataset_ids:
    params.map = {did: rtypes.rlong(ds_id)}
    ds_annos = find_objects_by_query(LINKED_ANNOS_BY_DS_QUERY, params, True)
    anno_ids = [i.id.val for i in ds_annos]
    anno_ids = list(set(anno_ids))  # remove any duplicates
#     print anno_ids
    annotation_ids.extend(anno_ids)
try:
    open_omero_session()
except Exception as e:
    print "Existing session"
print SESSION

In [ ]:
global dataset_ids, annotation_ids
print dataset_ids
print annotation_ids
# https://github.com/openmicroscopy/omero-example-scripts/blob/master/metadata_scripts/Metadata_Search.py
close_omero_session()
# destroy_omero_session()